In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm import create_model
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import wandb
from zipfile import ZipFile
from google.colab import files

In [ ]:
uploaded = files.upload()  # Upload your Task_A.zip here
with ZipFile("Task_A.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")


Saving Task_A.zip to Task_A.zip


In [ ]:
DATASET_ROOT = "/content/Task_A"


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
train_transform = A.Compose([
    A.RandomResizedCrop(size=(300, 300), scale=(0.8, 1.0)),
    A.HorizontalFlip(p=0.5),
    A.MotionBlur(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussNoise(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=224, width=224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.class_to_idx = {"female": 0, "male": 1}
        for class_name in ["female", "male"]:
            class_dir = os.path.join(root_dir, class_name)
            for fname in os.listdir(class_dir):
                self.samples.append((os.path.join(class_dir, fname), self.class_to_idx[class_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        image = np.array(image)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]
        return image, torch.tensor(label).float()


In [ ]:
train_dir = os.path.join(DATASET_ROOT, "train")
val_dir = os.path.join(DATASET_ROOT, "val")

if not os.path.isdir(train_dir) or not os.path.isdir(val_dir):
    raise FileNotFoundError("Dataset folders not found. Ensure your ZIP has Preprocessed_Task_A/train/ and Preprocessed_Task_A/val/ folders.")

train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset = CustomImageDataset(val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)


In [ ]:
model = create_model("efficientnet_b3", pretrained=True, num_classes=1, drop_rate=0.3)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-4, steps_per_epoch=len(train_loader), epochs=10)
scaler = torch.cuda.amp.GradScaler()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

/tmp/ipython-input-11-1115844922.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
best_acc = 0
epochs = 10
early_stop_counter = 0
early_stop_patience = 3

for epoch in range(epochs):
    model.train()
    train_loss, correct, total = 0, 0, 0
    loop = tqdm(train_loader)

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device).unsqueeze(1)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        preds = torch.sigmoid(outputs).detach().cpu().numpy() > 0.5
        correct += (preds == labels.cpu().numpy()).sum()
        total += labels.size(0)
        train_loss += loss.item() * labels.size(0)

        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        loop.set_postfix(loss=train_loss/total, acc=correct/total)

    scheduler.step()

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)

            preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
            val_correct += (preds == labels.cpu().numpy()).sum()
            val_total += labels.size(0)
            val_loss += loss.item() * labels.size(0)

    val_acc = val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model_efficientnetb3.pth")
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= early_stop_patience:
            print("Early stopping triggered.")
            break

print("Best Validation Accuracy:", best_acc)


  0%|          | 0/121 [00:00<?, ?it/s]

/tmp/ipython-input-12-2649213529.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Validation Accuracy: 0.7796


  0%|          | 0/121 [00:00<?, ?it/s]

Validation Accuracy: 0.8318


...

Validation Accuracy: 0.8791


  0%|          | 0/121 [00:00<?, ?it/s]

Validation Accuracy: 0.8744


  0%|          | 0/121 [00:00<?, ?it/s]

Validation Accuracy: 0.8791
Early stopping triggered.
Best Validation Accuracy: 0.8791469194312796


In [ ]:

def predict_on_validation_with_filenames(val_loader, model_path):
    model.load_state_dict(torch.load(model_path))
    model.eval()

    predictions = []

    with torch.no_grad():
        for batch in val_loader:
            if len(batch) == 3:
                img, _, img_paths = batch
            elif len(batch) == 2:
                img, img_paths = batch

            img = img.to(device)
            outputs = model(img)
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            for path, prob in zip(img_paths, probs):
                predictions.append((path, prob, "male" if prob > 0.5 else "female"))

    return predictions

val_predictions = predict_on_validation_with_filenames(val_loader, "best_model_efficientnetb3.pth")

for path, prob, pred_class in val_predictions[:10]:  # Display first 10 predictions
    print(f"{path} => {pred_class} ({prob:.3f})")


0.0 => female (0.005)
0.0 => male (0.999)
0.0 => male (0.976)
0.0 => male (0.997)
0.0 => male (0.855)
0.0 => female (0.000)
0.0 => female (0.014)
0.0 => female (0.145)
0.0 => female (0.001)
0.0 => male (0.927)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def calculate_metrics(loader, model, device):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.cpu().numpy()
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten() > 0.5

            y_true.extend(labels)
            y_pred.extend(preds)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return accuracy, precision, recall, f1


val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_loader, model, device)

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1-Score: {val_f1:.4f}")


Validation Accuracy: 0.8791
Validation Precision: 0.9056
Validation Recall: 0.9504
Validation F1-Score: 0.9275
